In [2]:
import mediapipe as mp
import cv2

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [63]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow('Raw Webcam Feed', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [64]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = holistic.process(image)
        
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [4]:
import csv
import os
import numpy as np

In [66]:
num_coords = len(results.pose_landmarks.landmark)
num_coords

33

In [67]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [68]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [40]:
class_name = "Bound Angle"

In [41]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        results = holistic.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
        
        # Export coordinates
        try:
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            row = pose_row
            row.insert(0, class_name)

            with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)

        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [69]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [70]:
df = pd.read_csv('coords.csv')

In [71]:
x = df.drop('class', axis=1)
y = df['class']

In [72]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1234)

ValueError: With n_samples=0, test_size=0.3 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [73]:
y_test

876            Plank
646       Warrior ll
1781      Warrior ll
499             Tree
1986    Child's Pose
            ...     
652       Warrior ll
202          Goddess
367             Tree
281             Tree
2803    Child's Pose
Name: class, Length: 992, dtype: object

In [5]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [75]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier())
}

In [10]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(x_train, y_train)
    fit_models[algo] = model

NameError: name 'pipelines' is not defined

In [77]:
fit_models['rf'].predict(x_test)

array(['Plank', 'Warrior ll', 'Warrior ll', 'Tree', "Child's Pose",
       'Bound Angle', "Child's Pose", "Child's Pose", 'Goddess',
       'Warrior ll', "Child's Pose", "Child's Pose", 'Warrior ll',
       'Plank', 'Plank', 'Plank', 'Plank', 'Goddess', 'Plank', 'Goddess',
       'Bound Angle', 'Bound Angle', 'Goddess', 'Plank', 'Plank',
       "Child's Pose", 'Plank', 'Goddess', "Child's Pose", "Child's Pose",
       'Tree', 'Plank', 'Bound Angle', 'Bound Angle', 'Plank',
       "Child's Pose", 'Bound Angle', 'Plank', "Child's Pose", 'Tree',
       "Child's Pose", 'Tree', 'Goddess', 'Warrior ll', 'Tree',
       "Child's Pose", 'Plank', 'Warrior ll', 'Tree', 'Bound Angle',
       'Bound Angle', 'Tree', 'Plank', 'Tree', 'Tree', 'Bound Angle',
       'Tree', 'Warrior ll', "Child's Pose", 'Plank', 'Plank',
       'Warrior ll', 'Warrior ll', "Child's Pose", "Child's Pose", 'Tree',
       'Plank', "Child's Pose", "Child's Pose", "Child's Pose",
       'Warrior ll', 'Goddess', 'Plank', 'Boun

In [6]:
from sklearn.metrics import accuracy_score
import pickle

In [83]:
for algo, model in fit_models.items():
    yhat = model.predict(x_test)
    print(algo, accuracy_score(y_test, yhat))

lr 1.0
rc 1.0
rf 0.9979838709677419


In [8]:
with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

NameError: name 'fit_models' is not defined

In [15]:
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

EOFError: Ran out of input

In [13]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        results = holistic.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
        
        # Export coordinates
        try:
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            row = pose_row

            x = pd.DataFrame([row])
            body_language_class = model.predict(x)[0]
            body_language_prob = model.predict_proba(x)[0]
            print(body_language_class, body_language_prob)

            cv2.rectangle(image, (0,0), (300, 60), (245, 117, 16), -1)

            cv2.putText(image, 'CLASS'
                        , (95, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class
                        , (90, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            cv2.putText(image, 'PROB'
                        , (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()